In [ ]:
#JSON 파일의 형식을 필요한 형식에 맞게 가져옵니다
import json

with open('./jira_test_cases_json/COMMONR-380.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    print(data)

{'expand': 'renderedFields,names,schema,operations,editmeta,changelog,versionedRepresentations', 'id': '69853', 'self': 'https://jira.suprema.co.kr/rest/api/2/issue/69853', 'key': 'COMMONR-380', 'fields': {'resolution': None, 'customfield_10510': None, 'customfield_10500': '0.0', 'customfield_10501': None, 'customfield_10502': None, 'customfield_12802': None, 'customfield_12804': None, 'customfield_10505': None, 'customfield_12803': None, 'customfield_10506': None, 'customfield_12806': None, 'customfield_10507': None, 'customfield_12805': None, 'customfield_10508': None, 'customfield_12808': None, 'customfield_10509': None, 'lastViewed': None, 'customfield_10180': None, 'customfield_10181': None, 'customfield_10182': None, 'customfield_10183': None, 'customfield_10184': {'self': 'https://jira.suprema.co.kr/rest/api/2/customFieldOption/10173', 'value': 'Major', 'id': '10173'}, 'customfield_10185': {'self': 'https://jira.suprema.co.kr/rest/api/2/customFieldOption/10220', 'value': 'Medium

In [ ]:
# --- 여기서부터 필요한 형식으로 만드는 과정 ---

# 1. 새로운 딕셔셔리를 만들어 필요한 정보만 추출하여 담기
#    .get()을 사용하면 키가 없을 때 오류 대신 None을 반환해 더 안전합니다.
required_format = {
    'issue_key': data.get('key'),
    'summary': data.get('fields', {}).get('summary'),
    'test_steps': data.get('fields', {}).get('customfield_10004', {}).get('steps', {})
}

# 2. 'test_steps' 리스트를 가공하여 필요한 열만 추출합니다.
original_steps = required_format['test_steps']
processed_steps = []

# for 반복문을 사용한 방법
for step_item in original_steps:
    new_step = {
        'step': step_item.get('step'),
        'data': step_item.get('data'),
        'result': step_item.get('result'),
    }
    processed_steps.append(new_step)

# 3. 가공된 새 리스트로 'test_steps' 값을 교체합니다.
required_format['test_steps'] = processed_steps
formatted_json_string = json.dumps(required_format, indent=2, ensure_ascii=False)
print(formatted_json_string)


{
  "issue_key": "COMMONR-380",
  "summary": "[BSX지원기능][BDP-954]Authentication - Door Control Admin ",
  "test_steps": [
    {
      "index": 1,
      "step": "1. Device> 관리자 설정",
      "data": "1. Master Password 설정\n2. 전체 관리자 설정\n3. 장치 설정 관리자 설정\n4. 사용자 관리자 설정",
      "result": "1. 사용자가 선택한 관리자로 설정되어야 한다.\n> 관리자에 대해 총 10명으로 제한(Old Version)\n  ex) 전체관리자 10명을 만들경우 다른 관리자에 대해 설정할 수 없음.\n> 관리자에 대해 총 1000명으로 제한(New Version)\n> Master Password는 1명 Only"
    },
    {
      "index": 2,
      "step": "[전체 관리자 미설정 상태]\n1. 장치 내에서 사용자에 사용자관리자 & 설정 관리자 설정\n2. BioStar X에서 장치>고급에서 사용자관리자 & 설정 관리자 설정\n3. 설정한 관리자 및 일반 사용자로 출입문 제어 진입 시도",
      "data": "1. CE 인증 장치\n2. CE 미인증 장치",
      "result": "1. 전체 관리자가 없는 상태에서 장치 설정 관리자로 출입문 제어 진입시도시 바로 진입이 가능해야 한다.\n2. 사용자관리자&일반 사용자의 경우 해당 메뉴에 진입할 수 없으므로 관리자 인증에 대한 오류 메세지를 표시한다.\n3. CE 미인증 장치 BioStar에서 전체관리자 미 설정 되어 있는 상태에서 사용자&설정 관리자 설정 시 BioStar 오류 메시지 출력"
    },
    {
      "index": 3,
      "step": "1. Device> 전체관리자 설정\n2. 사용자 관리자 & 설정관리자 설정\n3. 설정한 관리자로 출입

In [18]:
#필요한 json 파일을 통해서 임베딩 모델을 통해 임베딩 시작
%pip install sentence-transformers
# 1. 임베딩할 문장들을 담을 리스트 준비
sentences_to_embed = []
test_steps_list = formatted_json_string.get('test_steps', [])

# 2. 각 테스트 스텝을 순회하며 정보 추출 및 가공
for step_item in test_steps_list:
    # 각 필드의 값을 가져옵니다.
    step = step_item.get('step', '')
    step_data = step_item.get('data', '')
    step_result = step_item.get('result', '')

    # 전처리: 줄바꿈 문자(\n)를 공백으로 바꿔서 한 줄로 만듭니다.
    step_data_cleaned = step_data.replace('\n', ' ')
    step_result_cleaned = step_result.replace('\n', ' ')

    # f-string을 사용해 모든 정보를 하나의 문장으로 합칩니다.
    sentence = f"절차: {step}, 데이터: {step_data_cleaned}, 예상 결과: {step_result_cleaned}"
    sentences_to_embed.append(sentence)

# 3. 가공된 최종 문장 확인
print("--- 임베딩될 문장 ---")
print(sentences_to_embed[0])
print("-" * 25)

#임베딩 모델을 통해서 임베딩
try:
    from sentence_transformers import SentenceTransformer
    # 한국어에 특화된 공개 모델 중 하나를 사용합니다.
    # 처음 실행 시 모델을 다운로드하므로 시간이 걸릴 수 있습니다.
    model = SentenceTransformer('jinaai/jina-embeddings-v2-base-ko')
    
    embedding_vector_hf = model.encode(text_to_embed)

    print("\n✅ [방법 2] Hugging Face 임베딩 성공")
    print(f"벡터 차원: {len(embedding_vector_hf)}")
    # print(embedding_vector_hf[:5]) # 벡터의 앞 5개 값만 출력
except Exception as e:
    print(f"❌ [방법 2] Hugging Face 임베딩 실패: {e}")

  Using cached sentence_transformers-5.0.0-py3-none-any.whl.metadata (16 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached torch-2.7.1-cp312-none-macosx_11_0_arm64.whl.metadata (29 kB)
  Using cached scipy-1.16.0-cp312-cp312-macosx_14_0_arm64.whl.metadata (61 kB)
  Using cached pillow-11.3.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.0 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached numpy-2.3.1-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp312-cp312-macosx_11_0_arm64.whl.metadata (40 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached tokenizers-0.21.2-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached safetensors-0.5.3-cp38-abi3-macosx_11_0_arm64.whl.metadata (3.8 kB)
  Using cached hf_xet-1.1.5-cp37-abi3-macosx_11_0_arm64.whl.metadata (879 bytes)
  Usin

AttributeError: 'str' object has no attribute 'get'